In [ ]:
pip install parsons

In [10]:
import pandas as pd
from parsons import VAN

van =VAN(db='MyCampaign',api_key = "6a0bbb83-e829-4095-5f81-f15ab2c331a6")

In [ ]:
van.get_survey_questions(statuses=['Active'],name='Host Interest')


survey_questions INFO Found 1 survey questions.


surveyQuestionId,type,cycle,name,mediumName,shortName,scriptQuestion,status,responses
455016,Action,2022,Host Interest,Host Int,HI,Is this volunteer interested in hosting an event?,Active,"[{'surveyResponseId': 1853615, 'name': 'Yes', 'mediumName': 'Yes', 'shortName': 'Y'}, {'surveyResponseId': 1853616, 'name': 'No', 'mediumName': 'No', 'shortName': 'N'}]"


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving vol_signups - vol_signups.csv to vol_signups - vol_signups.csv


In [94]:
#create DF
signups=pd.read_csv('vol_signups - vol_signups.csv')
signups.head()

,LastName,FirstName,MiddleName,Suffix,Address,City,State/Province,Zip/Postal,Zip4,CountryCode,Preferred Phone,PreferredEmail,Age,Willing To Host a House Party
0,Ennis,James,P,NaN,5227 Hayden Woods Ln,Hilliard,OH,43026,7473.0,US,937-640-1818,James.Ennis@gmail.net,80,y
1,Collamer,Elaine,NaN,NaN,7302 Wethersfield Dr,West Chester,OH,45069,5535.0,US,937-748-5202,Elaine.Collamer@prodigy.org,54,n
2,Armour,Christine,Le,NaN,6854 Chablis Ct,Mentor,OH,44060,4071.0,US,440.205.1243,Christine.Armour@gmail.net,64,y
3,Nelson,Judith,C,NaN,6020 Joseph Dr,Oxford,OH,45056,1524.0,US,513-523-4478,Judith.Nelson@comcast,72,n
4,Kam,Frankie,K,NaN,986 Pinnacle Club Dr,Grove City,OH,43123,2585.0,US,614-871-2917,Frankie.Kam@hotmail.,75,y


In [95]:
print(signups.index)
signups.columns

RangeIndex(start=0, stop=1043, step=1)


Index(['LastName', 'FirstName', 'MiddleName', 'Suffix', 'Address', 'City',
       'State/Province', 'Zip/Postal', 'Zip4', 'CountryCode',
       'Preferred Phone', 'PreferredEmail', 'Age ',
       'Willing To Host a House Party'],
      dtype='object')

In [96]:
#column renaming
signups = signups.rename(columns={'Zip/Postal':'Zip5','PreferredEmail':'Email','Willing To Host a House Party':'HostInterest'})
signups.head()

,LastName,FirstName,MiddleName,Suffix,Address,City,State/Province,Zip5,Zip4,CountryCode,Preferred Phone,Email,Age,HostInterest
0,Ennis,James,P,NaN,5227 Hayden Woods Ln,Hilliard,OH,43026,7473.0,US,937-640-1818,James.Ennis@gmail.net,80,y
1,Collamer,Elaine,NaN,NaN,7302 Wethersfield Dr,West Chester,OH,45069,5535.0,US,937-748-5202,Elaine.Collamer@prodigy.org,54,n
2,Armour,Christine,Le,NaN,6854 Chablis Ct,Mentor,OH,44060,4071.0,US,440.205.1243,Christine.Armour@gmail.net,64,y
3,Nelson,Judith,C,NaN,6020 Joseph Dr,Oxford,OH,45056,1524.0,US,513-523-4478,Judith.Nelson@comcast,72,n
4,Kam,Frankie,K,NaN,986 Pinnacle Club Dr,Grove City,OH,43123,2585.0,US,614-871-2917,Frankie.Kam@hotmail.,75,y


In [98]:
#email can work as PK
signups[signups['Email'].isna()] #dedupping here cause no nulls

,LastName,FirstName,MiddleName,Suffix,Address,City,State/Province,Zip5,Zip4,CountryCode,Preferred Phone,Email,Age,HostInterest


In [99]:
#drop duplicated based on emails
signups[signups['Email'].duplicated(keep=False)].sort_values(by=['Email']).head(15)

,LastName,FirstName,MiddleName,Suffix,Address,City,State/Province,Zip5,Zip4,CountryCode,Preferred Phone,Email,Age,HostInterest
5,Davis,Alex,NaN,Jr,24986 Buckthorn Rd,Bedford Hts,OH,44146,4814.0,US,440.521.7092,Alex.Davis@hotmail.com,66,y
1039,Davis,Alex,NaN,Jr,24986 Buckthorn Rd,Bedford Hts,OH,44146,4814.0,US,440.521.7092,Alex.Davis@hotmail.com,66,y
758,Morez,Betty,Jean,NaN,676 Kimball Pl,Columbus,OH,43205,2862.0,US,(614) 914-6077,Betty.Morez@prodigy.com,66,n
15,Morez,Betty,Jean,NaN,676 Kimball Pl,Columbus,OH,43205,2862.0,US,(614) 914-6077,Betty.Morez@prodigy.com,66,n
770,Kenney,Carl,David,NaN,906 Villas Dr,Pataskala,OH,43062,8043.0,US,(740) 587-5287,Carl.Kenney@hotmail.com,84,y
613,Kenney,Carl,David,NaN,906 Villas Dr,Pataskala,OH,43062,8043.0,US,(740) 587-5287,Carl.Kenney@hotmail.com,84,y
380,Beutler,Carol,I,NaN,2030 Dickinson St Ste A,Fremont,OH,43420,5010.0,US,567.280.9068,Carol.Beutler@prodigy.com,88,n
357,Beutler,Carol,I,NaN,2030 Dickinson St Ste A,Fremont,OH,43420,5010.0,US,567.280.9068,Carol.Beutler@prodigy.com,88,n
49,Bednarski,Cheryl,L,NaN,5200 Heather Field Cir Apt A1,Parma,OH,44134,7906.0,US,440-887-7647,Cheryl.Bednarski@hotmail.com,55,n
103,Bednarski,Cheryl,L,NaN,5200 Heather Field Cir Apt A1,Parma,OH,44134,7906.0,US,440-887-7647,Cheryl.Bednarski@hotmail.com,55,n


In [100]:
print(signups.shape)
signups = signups.drop_duplicates(subset='Email')

(1043, 14)


In [103]:
#first email verification check

from validate_email import validate_email

signups['is_valid_email']=signups['Email'].apply(lambda x: validate_email(x))

signups.head(5)


,LastName,FirstName,MiddleName,Suffix,Address,City,State/Province,Zip5,Zip4,CountryCode,Preferred Phone,Email,Age,HostInterest,is_valid_email
0,Ennis,James,P,NaN,5227 Hayden Woods Ln,Hilliard,OH,43026,7473.0,US,937-640-1818,James.Ennis@gmail.net,80,y,True
1,Collamer,Elaine,NaN,NaN,7302 Wethersfield Dr,West Chester,OH,45069,5535.0,US,937-748-5202,Elaine.Collamer@prodigy.org,54,n,True
2,Armour,Christine,Le,NaN,6854 Chablis Ct,Mentor,OH,44060,4071.0,US,440.205.1243,Christine.Armour@gmail.net,64,y,True
3,Nelson,Judith,C,NaN,6020 Joseph Dr,Oxford,OH,45056,1524.0,US,513-523-4478,Judith.Nelson@comcast,72,n,True
4,Kam,Frankie,K,NaN,986 Pinnacle Club Dr,Grove City,OH,43123,2585.0,US,614-871-2917,Frankie.Kam@hotmail.,75,y,False


In [105]:
#create new df with valid emails
signups_VEmails=signups[signups['is_valid_email']== True]

In [107]:
#bad phones fix

import re
import numpy as np

def get_clean_phone(bad_phone):
  clean_phone = re.sub('\((\d{3})\)(\d{3})\-(\d{4})', '\g<1>\g<2>\g<3>', bad_phone)
  if clean_phone== bad_phone:
    clean_phone1 = re.sub('(\d{3})\-(\d{3})\-(\d{4})', '\g<1>\g<2>\g<3>', bad_phone)
    if clean_phone1 == bad_phone:
      clean_phone2 = re.sub('(\d{3})\.(\d{3})\.(\d{4})', '\g<1>\g<2>\g<3>', bad_phone)
      if clean_phone2 == bad_phone:
        clean_phone3= re.sub('\((\d{3})\)\s(\d{3})\-(\d{4})', '\g<1>\g<2>\g<3>', bad_phone)
        if clean_phone3 == bad_phone:
          clean_phone4 = np.NAN
          return clean_phone4
        return clean_phone3
      return clean_phone2
    return clean_phone1
  return clean_phone

In [108]:
#clean phones
signups_VEmails['phone']=signups_VEmails['Preferred Phone'].apply(get_clean_phone)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [109]:
#drop columns of no use
signups_cleaned=signups_VEmails.drop(columns={'is_valid_email','Preferred Phone'})

In [114]:
signups_cleaned.head(5)

,LastName,FirstName,MiddleName,Suffix,Address,City,State/Province,Zip5,Zip4,CountryCode,Email,Age,HostInterest,phone
0,Ennis,James,P,NaN,5227 Hayden Woods Ln,Hilliard,OH,43026,7473.0,US,James.Ennis@gmail.net,80,y,9376401818
1,Collamer,Elaine,NaN,NaN,7302 Wethersfield Dr,West Chester,OH,45069,5535.0,US,Elaine.Collamer@prodigy.org,54,n,9377485202
2,Armour,Christine,Le,NaN,6854 Chablis Ct,Mentor,OH,44060,4071.0,US,Christine.Armour@gmail.net,64,y,4402051243
3,Nelson,Judith,C,NaN,6020 Joseph Dr,Oxford,OH,45056,1524.0,US,Judith.Nelson@comcast,72,n,5135234478
5,Davis,Alex,NaN,Jr,24986 Buckthorn Rd,Bedford Hts,OH,44146,4814.0,US,Alex.Davis@hotmail.com,66,y,4405217092


In [115]:
#Kareem Jones fix eamil

def normalize_email_address(row):
  email = row
  if '@' not in email:
    return 'X' # Malformed email address :c
  name, domain = email.split('@')
  if '.' not in domain:
    return 'X' # Malformed email address :c
  domain = domain.strip('.')
  if '.' not in domain:
    return 'X' # Malformed email address :c
  provider, tld = domain.split('.', maxsplit=1)
  provider = provider.strip('.')
  tld = tld.strip('.')
  if not provider:
    return 'X' # Malformed email address :c
  if not tld:
    return 'X' # Malformed email address :c
  if tld == 'c':
    tld = 'com'
  if tld == 'orgg':
    tld = 'org'
  return f'{name}@{provider}.{tld}'

In [116]:
#create new emails since email verification didn't work
signups_cleaned['Email2']=signups_cleaned['Email'].apply(normalize_email_address)

In [117]:
signups_cleaned.head(5)

,LastName,FirstName,MiddleName,Suffix,Address,City,State/Province,Zip5,Zip4,CountryCode,Email,Age,HostInterest,phone,Email2
0,Ennis,James,P,NaN,5227 Hayden Woods Ln,Hilliard,OH,43026,7473.0,US,James.Ennis@gmail.net,80,y,9376401818,James.Ennis@gmail.net
1,Collamer,Elaine,NaN,NaN,7302 Wethersfield Dr,West Chester,OH,45069,5535.0,US,Elaine.Collamer@prodigy.org,54,n,9377485202,Elaine.Collamer@prodigy.org
2,Armour,Christine,Le,NaN,6854 Chablis Ct,Mentor,OH,44060,4071.0,US,Christine.Armour@gmail.net,64,y,4402051243,Christine.Armour@gmail.net
3,Nelson,Judith,C,NaN,6020 Joseph Dr,Oxford,OH,45056,1524.0,US,Judith.Nelson@comcast,72,n,5135234478,X
5,Davis,Alex,NaN,Jr,24986 Buckthorn Rd,Bedford Hts,OH,44146,4814.0,US,Alex.Davis@hotmail.com,66,y,4405217092,Alex.Davis@hotmail.com


In [118]:
#drop broken emails
signups_cleaned2=signups_cleaned[signups_cleaned['Email2'] != 'X']

In [119]:
signups_cleaned2.head(5)

,LastName,FirstName,MiddleName,Suffix,Address,City,State/Province,Zip5,Zip4,CountryCode,Email,Age,HostInterest,phone,Email2
0,Ennis,James,P,NaN,5227 Hayden Woods Ln,Hilliard,OH,43026,7473.0,US,James.Ennis@gmail.net,80,y,9376401818,James.Ennis@gmail.net
1,Collamer,Elaine,NaN,NaN,7302 Wethersfield Dr,West Chester,OH,45069,5535.0,US,Elaine.Collamer@prodigy.org,54,n,9377485202,Elaine.Collamer@prodigy.org
2,Armour,Christine,Le,NaN,6854 Chablis Ct,Mentor,OH,44060,4071.0,US,Christine.Armour@gmail.net,64,y,4402051243,Christine.Armour@gmail.net
5,Davis,Alex,NaN,Jr,24986 Buckthorn Rd,Bedford Hts,OH,44146,4814.0,US,Alex.Davis@hotmail.com,66,y,4405217092,Alex.Davis@hotmail.com
6,Oconnor,Alice,C,NaN,415 Bond Pl Apt 3C,Cincinnati,OH,45206,1878.0,US,Alice.Oconnor@prodigy.com,88,y,5132816116,Alice.Oconnor@prodigy.com


In [ ]:
#apply VAndIDS
vanids = []
for index, row in signups_cleaned2.iterrows():
    try:
        vanid = van.find_person(first_name = row.FirstName, last_name = row.LastName, email = row.Email2)['vanId']
    except:
        vanid = None
    vanids.append(vanid)

In [ ]:
#check if length works
len(vanids)

In [ ]:
#apply Vanids
signups_cleaned2['vanid']=vanids

In [ ]:
signups_cleaned2.head(5)

In [125]:
#translate y and n into response ID

sqs=[]

for x in signups_cleaned2['HostInterest']:
  if x=='y':
    z=1853615
  elif x=='n':
    z=1853616
  else:
    None
  sqs.append(z)


In [ ]:
#apply the survery codes
signups_cleaned2['SQ']=sqs

In [127]:
#drop contacts we couldn't match
signupds_cleaned3=signups_cleaned2.dropna(subset=['vanid'])

In [128]:
#check to see if VanID na drop worked
signupds_cleaned3.head(2)

,LastName,FirstName,MiddleName,Suffix,Address,City,State/Province,Zip5,Zip4,CountryCode,Email,Age,HostInterest,phone,Email2,vanid,SQ
0,Ennis,James,P,NaN,5227 Hayden Woods Ln,Hilliard,OH,43026,7473.0,US,James.Ennis@gmail.net,80,y,9376401818,James.Ennis@gmail.net,103051972.0,1853615
1,Collamer,Elaine,NaN,NaN,7302 Wethersfield Dr,West Chester,OH,45069,5535.0,US,Elaine.Collamer@prodigy.org,54,n,9377485202,Elaine.Collamer@prodigy.org,103054139.0,1853616


In [ ]:
#iteror to apply surver question use van.get_canvass_responses_X_types to get Contact type, input type
# use get_survey_x to get question_id and response_id

for index, row in signups_cleaned2.iterrows():
    try:
      job_id=van.apply_survey_response(id= row.vanid, survey_question_id=455016,survey_response_id=row.SQ, contact_type_id=13,input_type_id=5)
    except:
        print('No')

In [ ]:
van.get_survey_question